<a href="https://colab.research.google.com/github/JimKing100/nfl-test/blob/master/Rookies_Final_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Installs
%%capture
!pip install category_encoders==2.0.0

In [0]:
# Import libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
import category_encoders as ce
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor

import category_encoders as ce

In [0]:
player_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-actuals/players_full.csv')
kickers_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-actuals/actuals_rookie_kickers.csv')
offense_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-actuals/actuals_rookie_offense.csv')
rookie_kickers_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-actuals/actuals_rookie2019_kickers.csv')
rookie_offense_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-actuals/actuals_rookie2019_offense.csv')
actuals_df = pd.concat([kickers_df, offense_df], ignore_index=True)
actuals_rookie_df = pd.concat([rookie_kickers_df, rookie_offense_df])

In [4]:
player_df.head()

,player,first,last,name,position1,start,cteam,height,weight,dob,forty,bench,vertical,broad,shuttle,cone,arm,hand,dpos,col,dv,jnum,dcp
0,SF,San Francisco,49ers,San Francisco 49ers,DF,2000,SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CHI,Chicago,Bears,Chicago Bears,DF,2000,CHI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CIN,Cincinnati,Bengals,Cincinnati Bengals,DF,2000,CIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BUF,Buffalo,Bills,Buffalo Bills,DF,2000,BUF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DEN,Denver,Broncos,Denver Broncos,DF,2000,DEN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
actuals_df.head()

,player,first,last,name,position1,game1,game2,game3,game4,game5,game6,game7,game8,game9,game10,game11,game12,game13,game14,game15,game16
0,Adam Vinatieri,Adam,Vinatieri,Adam Vinatieri,K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Matt Bryant,Matt,Bryant,Matt Bryant,K,7.0,14.0,10.0,1.0,3.0,4.0,3.0,6.0,-1.0,13.0,2.0,10.0,10.0,12.0,8.0,4.0
2,Robbie Gould,Robbie,Gould,Robbie Gould,K,5.0,4.0,4.0,7.0,7.0,5.0,6.0,7.0,13.0,2.0,10.0,7.0,3.0,3.0,0.0,0.0
3,Mike Nugent,Mike,Nugent,Mike Nugent,K,0.0,6.0,8.0,3.0,0.0,6.0,2.0,8.0,2.0,15.0,3.0,13.0,10.0,3.0,13.0,0.0
4,Stephen Gostkowski,Stephen,Gostkowski,Stephen Gostkowski,K,5.0,5.0,0.0,7.0,8.0,4.0,8.0,9.0,5.0,5.0,7.0,8.0,16.0,7.0,10.0,13.0


In [6]:
actuals_rookie_df.head()

,player,first,last,name,position1,game1,game2,game3,game4,game5,game6,game7,game8,game9,game10,game11,game12,game13,game14,game15,game16
0,Matt Gay,Matt,Gay,Matt Gay,K,5.0,8.0,15.0,15.0,7.0,6.0,10.0,12.0,13.0,7.0,5.0,8.0,9.0,9.0,10.00,1.00
1,Chase McLaughlin,Chase,McLaughlin,Chase McLaughlin,K,8.0,5.0,10.0,5.0,11.0,6.0,14.0,10.0,1.0,7.0,10.0,0.0,0.0,0.0,0.00,0.00
2,Austin Seibert,Austin,Seibert,Austin Seibert,K,1.0,13.0,7.0,11.0,3.0,2.0,8.0,13.0,8.0,3.0,10.0,6.0,11.0,7.0,3.00,6.00
3,Joey Slye,Joey,Slye,Joey Slye,K,11.0,16.0,8.0,13.0,2.0,14.0,4.0,6.0,5.0,3.0,10.0,2.0,10.0,8.0,8.00,4.00
0,David Blough,David,Blough,David Blough,QB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.3,8.2,8.3,8.68,11.88


In [0]:
def create_final(df, df1):

    # The main code for iterating through the player list, calculating the points and adding the rows
    # to the final_df dataframe.
    column_names = ['player',
                    'position1',
                    'height',
                    'weight',
                    'forty',
                    'bench',
                    'vertical',
                    'broad',
                    'shuttle',
                    'cone',
                    'arm',
                    'hand',
                    'dpos',
                    'col',
                    'dv',
                    'points' 
                    ]

    player_list = df['player'].tolist()

    final_df = pd.DataFrame(columns = column_names)

    for player in player_list:

        position1 = df1['position1'].loc[(df1['name']==player)].iloc[0]
        height = df1['height'].loc[(df1['name']==player)].iloc[0]
        weight = df1['weight'].loc[(df1['name']==player)].iloc[0]
        forty = df1['forty'].loc[(df1['name']==player)].iloc[0]
        bench = df1['bench'].loc[(df1['name']==player)].iloc[0]
        vertical = df1['vertical'].loc[(df1['name']==player)].iloc[0]
        broad = df1['broad'].loc[(df1['name']==player)].iloc[0]
        shuttle = df1['shuttle'].loc[(df1['name']==player)].iloc[0]
        core = df1['cone'].loc[(df1['name']==player)].iloc[0]
        arm = df1['arm'].loc[(df1['name']==player)].iloc[0]
        hand = df1['hand'].loc[(df1['name']==player)].iloc[0]
        dpos = df1['dpos'].loc[(df1['name']==player)].iloc[0]
        college = df1['col'].loc[(df1['name']==player)].iloc[0]
        division = df1['dv'].loc[(df1['name']==player)].iloc[0]
        points = df.loc[(df['name']==player)].iloc[0, 5:21].sum()

        final_df = final_df.append({'player': player,
                                    'position1': position1,
                                    'height': height,
                                    'weight': weight,
                                    'forty': forty,
                                    'bench': bench,
                                    'vertical': vertical,
                                    'broad': broad,
                                    'shuttle': shuttle,
                                    'cone': core,
                                    'arm': arm,
                                    'hand': hand,
                                    'dpos': dpos,
                                    'col': college,
                                    'dv': division,
                                    'points': points 
                                     }, ignore_index=True)
    return final_df
  

In [0]:
train_df = create_final(actuals_df, player_df)
test_df = create_final(actuals_rookie_df, player_df)

In [9]:
train_df.tail()

,player,position1,height,weight,forty,bench,vertical,broad,shuttle,cone,arm,hand,dpos,col,dv,points
510,Steven Mitchell,WR,70.0,189.0,4.56,13.0,33.5,121.0,4.40,6.75,0.000,0.000,0.0,USC,Pacific 12,0.0
511,Trey Quinn,WR,71.0,203.0,4.46,17.0,33.5,104.0,4.17,6.91,30.750,10.125,256.0,SMU,American Athletic,13.5
512,Trent Sherfield,WR,72.0,200.0,4.48,19.0,32.5,125.0,4.20,6.90,0.000,0.000,0.0,Vanderbilt,Southeastern (SEC),27.0
513,Tre'Quan Smith,WR,74.0,203.0,4.49,12.0,37.5,130.0,4.50,6.97,33.375,9.500,91.0,Central Florida,American Athletic,74.7
514,Vyncint Smith,WR,75.0,195.0,4.38,0.0,39.5,130.0,4.27,6.89,0.000,0.000,0.0,Limestone,0,15.1


In [0]:
target = 'points'
X_train = train_df.drop(columns=[target])
y_train = train_df[target]
X_test = test_df.drop(columns=[target])
y_test = test_df[target]

In [0]:
# Split the initial train features and labels 80/20 into train and validate
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, train_size = 0.80, test_size = 0.20)

In [12]:
pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    XGBRegressor(n_estimators=200, n_jobs=-1)
)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_val)

# Print metrics for validation 
val_mse = mean_squared_error(y_val, y_pred)
val_rmse = np.sqrt(val_mse)
val_mae = mean_absolute_error(y_val, y_pred)
val_r2 = r2_score(y_val, y_pred)
print('Validation Mean Absolute Error:', val_mae)
print('Validation R^2:', val_r2)
print('\n')

ty_pred = pipeline.predict(X_test)

# Print metrics for test
test_mse = mean_squared_error(y_test, ty_pred)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, ty_pred)
test_r2 = r2_score(y_test, ty_pred)
print('Test Mean Absolute Error:', test_mae)
print('Test R^2:', test_r2)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[02:57:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Validation Mean Absolute Error: 37.29974764219765
Validation R^2: 0.3553737123040728


Test Mean Absolute Error: 33.951482366570886
Test R^2: 0.3920622246750518


In [13]:
# Add the prediction, difference and address to the final dataframe
final = test_df.copy()
final = final.reset_index()
final['prediction'] = ty_pred
final['difference'] = final['prediction'] - final['points']
final['prediction'] = final['prediction'].astype(int)
final['difference'] = final['difference'].astype(int)
print(final.shape)
final.tail()

(106, 19)


,index,player,position1,height,weight,forty,bench,vertical,broad,shuttle,cone,arm,hand,dpos,col,dv,points,prediction,difference
101,101,Spencer Schnell,WR,68.0,176.0,4.64,13.0,36.5,121.0,4.34,6.71,29.250,9.250,0.0,Illinois State,Missouri Valley,0.0,61,61
102,102,Steven Sims Jr.,WR,69.0,181.0,4.56,11.0,31.5,115.0,4.43,7.20,29.375,8.625,0.0,Kansas,Big 12,77.5,13,-63
103,103,Travis Fulgham,WR,74.0,215.0,4.55,15.0,36.5,126.0,4.22,6.84,33.625,9.500,184.0,Old Dominion,Conference USA (C-USA),0.0,4,4
104,104,Terry McLaurin,WR,72.0,208.0,4.35,18.0,37.5,125.0,4.15,7.01,31.500,9.125,76.0,Ohio State,Big Ten,137.9,57,-80
105,105,Ventell Bryant,WR,75.0,198.0,4.61,12.0,34.5,121.0,4.25,6.97,33.125,9.750,0.0,Temple,American Athletic,7.5,19,11
